## Algoritmos Genéticos: Problema del Viajero

Siguiendo los pasos de la teoria de Algoritmos Genéticos, has el algoritmo del problema del viajero.

- **Población Inicial:** 20
- **Número de Genes:** 21
- **Probabilidad de Cruce:** 0.9
- **Probabilidad de Mutación:** 0.1

Utiliza **`madrid_duracion.csv`** para este problema. Este **`.csv`** tiene el tiempo de distancia entre cada barrio de Madrid. El objetivo es hacer un algoritmo que pase por todos los barrios en el menor tiempo posible sin pasar por el mismo barrio 2 veces.

1. Define una función para inicializar la población (20 individuos).


2. Define una función para el calculo del **`fitness`**, esta función debe sumar el tiempo del barrio **`i`** al barrio **`j`**. El mejor **`fitness`** es el que tenga el valor más bajo (menor tiempo).


3. Define una función de selección de padres.


4. Define una función para el cruce de los padres.


5. Define una función para la mutación del/los hijo(s).


6. Define una función para reemplazar los elementos de la población.

In [ ]:
import numpy as np
import pandas as pd

import random

In [ ]:
df_tiempo = pd.read_csv("madrid_duracion.csv")

df_tiempo.index = df_tiempo.columns

df_tiempo

In [ ]:
df_tiempo = df_tiempo.applymap(lambda x : int(x.split(" ")[0]))

df_tiempo.head(3)

In [ ]:
# mat_tiempo.info()

In [ ]:
mat_tiempo = np.array(df_tiempo)

mat_tiempo

In [ ]:
map_barrios = {num : col for num, col in enumerate(df_tiempo.columns)}

map_barrios

In [ ]:
# Poblacion

# def generar_poblacion(n):
    
#     poblacion = list()
    
#     for i in range(n):
#         lista = list()

#         while len(lista) != len(dist):
#             barrio_aleatorio = np.random.randint(0, len(dist))

#             if barrio_aleatorio not in lista:
#                 lista.append(barrio_aleatorio)

#         poblacion.append(lista)
        
#     return poblacion

# %%time
# generar_poblacion(10000)

def generar_poblacion(n_poblacion, n_barrios):
    poblacion = [list(range(n_barrios)) for i in range(n_poblacion)]

    for i in range(len(poblacion)):
        np.random.shuffle(poblacion[i])

    return np.array(poblacion)

In [ ]:
# Fitness

def calculo_fitness(individuo, mat_tiempo):
        
    total = 0

    for i in range(len(individuo) - 1):
        total += mat_tiempo[int(individuo[i]), int(individuo[i + 1])]

    return total


def calculo_fitness(individuo, mat_tiempo):
    return sum([mat_tiempo[int(individuo[i]), int(individuo[i + 1])] for i in range(len(individuo) - 1)])

In [ ]:
# Torneo

def seleccion_torneo(poblacion, k, ganador = True):
    
    torneo = np.array(random.sample(population = list(poblacion), k = k))
                      
    lista_fitness = [calculo_fitness(individuo, mat_tiempo) for individuo in torneo]
    
    if ganador:
    
        menor_fitness_indice = np.argmin(lista_fitness)

        return torneo[menor_fitness_indice]
    
    else:
        
        mayor_fitness_indice = np.argmax(lista_fitness)

        return torneo[mayor_fitness_indice] 

In [ ]:
# Cruce

def cruce_orden(padre_1, padre_2, p_cruce):
    
    if np.random.random() < p_cruce:
        
        hijos = [np.zeros(len(padre_1)) - 1, np.zeros(len(padre_1)) - 1]
        
        for h in range(2):
        
            k = np.random.randint(8, len(padre_1)//2)

            indices_fijos = sorted(random.sample(population = range(len(padre_1)), k  = k))

            for i in range(len(indices_fijos)):
                hijos[h][indices_fijos[i]] = padre_1[indices_fijos[i]]

            indices_variables = list()

            for p in padre_2:
                if p not in hijos[h]:
                    indices_variables.append(p)

            # indices_variables = [p for p in padre_2 if p not in hijo_1]    

            j = 0
            for i in range(len(hijos[h])):
                if hijos[h][i] == -1:
                    hijos[h][i] = indices_variables[j]
                    j += 1
            
        return tuple(hijos)     
    else:
        return padre_1, padre_2

In [ ]:
# Mutacion

def mutacion(hijo, p_mutacion):
    if np.random.random() < p_mutacion:
        hijo_copia = hijo.copy()
        
        indice_1, indice_2 = random.sample(population = range(len(hijo)), k = 2)
        
        hijo_copia[indice_1] = hijo[indice_2]
        hijo_copia[indice_2] = hijo[indice_1]
        
        return hijo_copia
    
    else:
        return hijo

In [ ]:
# Mutacion

def mutacion1(hijo, p_mutacion):
    if np.random.random() < p_mutacion:
        hijo_copia = hijo.copy()
        
        indice_1, indice_2 = random.sample(population = range(1, len(hijo) - 1), k = 2)
        
        hijo_copia[indice_1] = hijo[indice_2]
        hijo_copia[indice_2] = hijo[indice_1]

        return hijo_copia
    
    else:
        return hijo

In [ ]:
# Reemplazo

def reemplazo_estacionario_torneo(poblacion, hijo_1, hijo_2):
    
    poblacion = list(poblacion)
    
    for i in range(2):
        remover = seleccion_torneo(poblacion, 4, False)
        for j in range(len(poblacion)):
            if all(poblacion[j] == remover):
                poblacion.pop(j)
                break
        
    poblacion.extend([hijo_1, hijo_2])
    
    return np.array(poblacion)

In [ ]:
n_poblacion = 20
n_genes = len(mat_tiempo)
p_cruce = 0.9
p_mutacion = 0.1

generaciones = 10_000

In [ ]:
%%time
# Algoritmo Genético

poblacion = generar_poblacion(n_poblacion, n_genes)

mejores_individuos = list()

for generacion in range(generaciones):
    
    # Selección Padres
    padre_1, padre_2 = seleccion_torneo(poblacion, 4), seleccion_torneo(poblacion, 4)
    
    # Cruce (Hijos)
    hijo_1, hijo_2 = cruce_orden(padre_1, padre_2, p_cruce)
    
    # Mutacion
    hijo_1 = mutacion(hijo_1, p_mutacion)
    hijo_2 = mutacion(hijo_2, p_mutacion)
    
    # Reemplazo
    poblacion = reemplazo_estacionario_torneo(poblacion, hijo_1, hijo_2)
    
    # Mejor Individuo
    mejores_individuos.append(seleccion_torneo(poblacion, n_poblacion))

In [ ]:
mejores_fitness = [calculo_fitness(individuo, mat_tiempo) for individuo in mejores_individuos]

mejores_individuos[np.argmin(mejores_fitness)]

In [ ]:
calculo_fitness(mejores_individuos[np.argmin(mejores_fitness)], mat_tiempo)/60

In [ ]:
for i in mejores_individuos[np.argmin(mejores_fitness)]:
    print(map_barrios[i])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (12, 8))

plt.plot(mejores_fitness, color = "red")
plt.show()

In [ ]:
############################################################################################################################

In [ ]:
# Problema Empieza y Acaba en el mismo punto

In [ ]:
poblacion = generar_poblacion(n_poblacion, n_genes)

# poblacion

In [ ]:
barrio = 4 # Chamartin

poblacion = list(poblacion)

for i in range(len(poblacion)):
    poblacion[i] = list(poblacion[i])
    poblacion[i].remove(barrio)
    poblacion[i].insert(0, barrio)
    poblacion[i].append(barrio)
    
poblacion

In [ ]:
mejores_individuos = list()

for generacion in range(generaciones):
    
    # Torneo
    padre_1 = seleccion_torneo(poblacion, 4)
    padre_2 = seleccion_torneo(poblacion, 4)
    
    # Cruce (Hijos)
    hijo_1, hijo_2 = cruce_orden(padre_1[1:-1], padre_2[1:-1], p_cruce)
    hijo_1, hijo_2 = list(hijo_1), list(hijo_2)
    hijo_1.insert(0, barrio)
    hijo_1.append(barrio)
    hijo_2.insert(0, barrio)
    hijo_2.append(barrio)
    
    # Mutacion
    hijo_1 = mutacion1(hijo_1, p_mutacion)
    hijo_2 = mutacion1(hijo_2, p_mutacion)
    
    # Reemplazo
    poblacion = reemplazo_estacionario_torneo(poblacion, hijo_1, hijo_2)
    
    # Mejor Individuo
    mejores_individuos.append(seleccion_torneo(poblacion, n_poblacion))

In [ ]:
mejores_fitness = [calculo_fitness(individuo, mat_tiempo) for individuo in mejores_individuos]

mejores_individuos[np.argmin(mejores_fitness)]

In [ ]:
calculo_fitness(mejores_individuos[np.argmin(mejores_fitness)], mat_tiempo)/60

In [ ]:
for i in mejores_individuos[np.argmin(mejores_fitness)]:
    print(map_barrios[i])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (12, 8))

plt.plot(mejores_fitness, color = "red")
plt.show()

In [ ]:
############################################################################################################################